## Generation-time branch attraction

This notebook focuses on (f) effects

In [1]:
# conda install ipcoal ipyrad -c conda-forge -c bioconda 

In [1]:
import numpy as np
import pandas as pd
import toytree
import toyplot, toyplot.svg
import ipcoal
import ipyrad.analysis as ipa

### Treestyle

In [15]:
ts = {
    'ts': 'p', 
    'layout': 'r',
    'height': 300,
    'node_sizes': 7,
    'node_style': {"stroke": "black", 'stroke-width': 2},
    'node_labels': False,
    'edge_type': 'c',
    'scalebar': False,
    'tip_labels_align': True,
    'scalebar': True,
}

### Starting simulation scenario with deep divergences

In [3]:
# get an ultrametric imbalanced tree
tree = toytree.rtree.imbtree(8, treeheight=20e6)
tree.draw(**ts);

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t56f134516f974cfea0a1a28c5cfbece3" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 260.0 300.0" width="260.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r0 r1 r2 r3 r4 r5 r6 r7 0 10000000 20000000

### Modify species tree params to make "variable N and g" tree (f)

In [4]:
# set gentime on nodes
ftree = tree.set_node_values(
    feature="g", 
    values={i: 10 for i in (3,4,5,10,11,12)},
    default=1,
)

# set Ne on nodes
ftree = ftree.set_node_values(
    feature="Ne", 
    values={i: 1e6 for i in (3,4,5,10,11,12)},
    default=10e6,
)

# set edge lengths to be in generations
ftree = ftree.set_node_values(
    feature="dist", 
    values={i: j.dist / j.g for (i, j) in ftree.idx_dict.items()}
)

# draw the tree
ftree.draw(**ts);

# save tree to file
ftree.write("./trees/f-tree.nwk")

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t765545ecaa3247f39b3a01f9848000e2" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 260.0 300.0" width="260.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r0 r1 r2 r3 r4 r5 r6 r7 0 10000000 20000000

### Simulate a concatenated sequence (chromosome)

In [12]:
# simulate a long chrom
model = ipcoal.Model(ftree, seed=123)
model.sim_loci(1, 1e6)

# save to db
model.write_loci_to_hdf5(name="f-concat", outdir="db")

# show the total number of genealogies
print('ngenealogies:', model.df.tidx.max())
print('mean len of gtree: {:.2f} bp'.format(model.df.nbps.mean()))

# show the locus (pretty high variation)
model.draw_seqview(0, 0, 50);

ngenealogies: 104930
mean len of gtree: 9.53 bp


<svg class="toyplot-canvas-Canvas" height="128.0px" id="t1580f9dbf100421c831e3dfde843ae83" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 1020.0 128.0" width="1020.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r0 r1 r2 r3 r4 r5 r6 r7

### Confirm concatenation is in the anomaly zone 

Here we aim to select a scenario that will lie in the 'anomaly zone', where concatenation will yield incorrect results but a proper MSC method should infer a correct result.

In [16]:
# init raxml inference
tool = ipa.treeslider(
    data="./db/f-concat.seqs.hdf5",
    name="f-concat",
    scaffold_idxs=0,
    inference_args={
        "f": "d",
        "N": 10,
        "p": 12345,
        "x": None,
        "T": 20,
    }
)
tool.ipcluster['threads'] = 20
tool.show_inference_command()

# infer concat tree
tool.run(auto=True, force=True)

/home/deren/miniconda3/envs/ipy/bin/raxmlHPC-PTHREADS-AVX2 -f d -T 20 -m GTRGAMMA -n ... -w ... -s ... -p 12345 -N 10
building database: nwindows=1; minsnps=1
[####################] 100% 0:00:27 | inferring trees 
tree_table written to /home/deren/gentime-attraction/notebooks/analysis-treeslider/f-concat.tree_table.csv


In [17]:
# re-load the newick from treeslider output
tw = pd.read_csv("./analysis-treeslider/f-concat.tree_table.csv")
etree = toytree.tree(tw.tree[0]).root("r7")

# draw the concat tree
etree.draw(**ts);

# save tree file
etree.write("./trees/f-concat.nwk")

<svg class="toyplot-canvas-Canvas" height="300.0px" id="td197fd6e09764813b08a552e6e40f560" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 260.0 300.0" width="260.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r0 r1 r2 r3 r4 r5 r6 r7 0.0 0.3 0.6

### Simulate unlinked genealogies

In [18]:
# simulate many unlinked genealogies (same as above)
model = ipcoal.Model(ftree, seed=123)
model.sim_trees(nloci=10000, nsites=1)

# load into a tree object and draw a few
mtre = toytree.mtree(model.df.genealogy)
mtre.draw();

# convert bl units to E(subst.)
for tre in mtre.treelist:
    for node in tre.idx_dict.values():
        node.dist = node.dist * 1e-8      
        
# write re-formatted trees to a tmp file
mtre.write("./trees/f-genealogies.nwk")

<svg class="toyplot-canvas-Canvas" height="200.0px" id="t8e294b12e5da4bdab9186f76a6c1e712" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 560.0 200.0" width="560.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r4 r2 r0 r6 r7 r3 r1 r5 r4 r2 r5 r7 r6 r1 r0 r3 r2 r1 r3 r0 r6 r7 r5 r4 r4 r3 r1 r0 r2 r7 r6 r5

### Confirm ASTRAL-genealogies is not an anomaly


In [19]:
# infer MSC sptree with ASTRAL
ast = ipa.astral(
    data=[i.write() for i in toytree.mtree("./trees/f-genealogies.nwk")],
    name="f-astral-genealogy",
)
ast.run()

# draw sptree
atree = toytree.tree(ast.tree).root("r7")
atree.draw(**ts);

# save tree file
atree.write("./trees/f-astral-genealogies.nwk")

[astral.5.7.1.jar]
inferred tree written to (/home/deren/gentime-attraction/notebooks/analysis-astral/f-astral-genealogy.tre)


<svg class="toyplot-canvas-Canvas" height="300.0px" id="t2745774e1c1c4f5c9e821557547719c4" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 260.0 300.0" width="260.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r1 r0 r2 r3 r4 r5 r6 r7 0 1 2

### Confirm SNAQ-genealogies is not an anomaly

In [20]:
# infer sptree
snaq0 = ipa.snaq(
    gtrees="./trees/f-genealogies.nwk",
    netin="./trees/f-astral-genealogies.nwk",
    name="f-snaq-genealogies-net0",
    nedges=0,
    nruns=10,
    nproc=10,
    seed=123,
)
snaq0.run()

# infer sptree
snaq1 = ipa.snaq(
    gtrees="./trees/f-genealogies.nwk",
    netin="./trees/f-astral-genealogies.nwk",
    name="f-snaq-genealogies-net1",
    nedges=1,
    nruns=10,
    nproc=10,
    seed=123,
)
snaq1.run()

using existing CF table: /home/deren/gentime-attraction/notebooks/analysis-snaq/f-snaq-genealogies-net0.CFs.csv
[SNAQ v.x.y]
[nproc = 10]
julia /home/deren/gentime-attraction/notebooks/analysis-snaq/f-snaq-genealogies-net0.jl
inferred network written to (/home/deren/gentime-attraction/notebooks/analysis-snaq/f-snaq-genealogies-net0.snaq)
using existing CF table: /home/deren/gentime-attraction/notebooks/analysis-snaq/f-snaq-genealogies-net1.CFs.csv
[SNAQ v.x.y]
[nproc = 10]
julia /home/deren/gentime-attraction/notebooks/analysis-snaq/f-snaq-genealogies-net1.jl
inferred network written to (/home/deren/gentime-attraction/notebooks/analysis-snaq/f-snaq-genealogies-net1.snaq)


In [21]:
snaq1.admix

{'H9': (['r0'], ['r4'], 0.5, {}, '0.035')}

In [22]:
# draw tree
stree = toytree.tree(snaq1.tree).root("r7")
stree.draw(**ts, admixture_edges=snaq1.admix.values());

# write network
stree.write("trees/f-snaq-genealogies.nwk")

<svg class="toyplot-canvas-Canvas" height="300.0px" id="td82579e015af42c5a704c0e7f99b66c7" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 260.0 300.0" width="260.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0.035 r0 r1 r2 r3 r4 r5 r6 r7 0 1 2

### Simulate loci
Do 10K loci and test a few range of locus lengths to allow examining length effect.

In [23]:
# simulate loci that are each Nbp in length
model = ipcoal.Model(ftree, seed=123)
model.sim_loci(nloci=10000, nsites=2e3)
model.write_loci_to_hdf5(name="f-10000l-2000s", outdir="db")
model.write_snps_to_hdf5(name="f-10000l-2000s", outdir="db")

# draw seqview
model.draw_seqview(0, 0, 50);

wrote 10000 loci to /home/deren/gentime-attraction/notebooks/db/f-10000l-2000s.seqs.hdf5
wrote 15010421 SNPs to /home/deren/gentime-attraction/notebooks/db/f-10000l-2000s.snps.hdf5


<svg class="toyplot-canvas-Canvas" height="128.0px" id="t13af98a31ba74424af3c0ff0a6cfaea6" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 1020.0 128.0" width="1020.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r0 r1 r2 r3 r4 r5 r6 r7

### Infer gene trees from loci


In [24]:
# init tree slider inference tool
tool = ipa.treeslider(
    data="./db/f-10000l-2000s.seqs.hdf5",
    name="f-10000l-2000s",
    scaffold_idxs=range(10000),
    inference_args={
        "N": 10,
        "f": "d",
        "p": 12345,
        "x": None,
        "T": 2,
    }
)

# infer all gene trees
tool.ipcluster['cores'] = 20
tool.ipcluster['threads'] = 2
tool.run(auto=True, force=True)

building database: nwindows=10000; minsnps=1
[####################] 100% 1:03:43 | inferring trees 
tree_table written to /home/deren/gentime-attraction/notebooks/analysis-treeslider/f-10000l-2000s.tree_table.csv


In [30]:
# load tree slider (gene tree inference) results
tw = pd.read_csv("./analysis-treeslider/f-10000l-2000s.tree_table.csv", index_col=0)

# load gene trees into toytree multitree object
mtre = toytree.mtree(tw.tree)
mtre.treelist = [i.root("r7") for i in mtre.treelist]
mtre.draw(ts='o', use_edge_lengths=False);

# write list of trees as a tmp file
print(len(mtre), "genetrees")
mtre.write("./trees/f-genetrees.nwk")

10000 genetrees


<svg class="toyplot-canvas-Canvas" height="200.0px" id="t35637c7de5b5447bb69d0181fd8c986c" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 560.0 200.0" width="560.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r5 r1 r0 r4 r2 r3 r6 r7 r0 r3 r2 r1 r5 r4 r6 r7 r3 r2 r1 r0 r4 r6 r5 r7 r5 r4 r3 r2 r0 r1 r6 r7

### Infer SNAQ network from inferred gene trees

In [26]:
snaq0 = ipa.snaq(
    gtrees="./trees/f-genetrees.nwk",
    netin="./trees/f-astral-genealogies.nwk",
    name="f-snaq-genetrees-net0",
    nedges=0,
    nruns=10,
    nproc=10,
)
snaq0.run()

snaq1 = ipa.snaq(
    gtrees="./trees/f-genetrees.nwk",
    netin="./trees/f-astral-genealogies.nwk",
    name="f-snaq-genetrees-net1",
    nedges=1,
    nruns=10,
    nproc=10,
)
snaq1.run()

[SNAQ v.x.y]
[nproc = 10]
julia /home/deren/gentime-attraction/notebooks/analysis-snaq/f-snaq-genetrees-net0.jl
inferred network written to (/home/deren/gentime-attraction/notebooks/analysis-snaq/f-snaq-genetrees-net0.snaq)
[SNAQ v.x.y]
[nproc = 10]
julia /home/deren/gentime-attraction/notebooks/analysis-snaq/f-snaq-genetrees-net1.jl
inferred network written to (/home/deren/gentime-attraction/notebooks/analysis-snaq/f-snaq-genetrees-net1.snaq)


In [27]:
snaq1.admix

{'H9': (['r3'], ['r2'], 0.5, {}, '0.477')}

In [32]:
# draw tree
stree = toytree.tree(snaq1.tree).root("r7")
stree.draw(**ts, admixture_edges=snaq1.admix.values());

# save tree
stree.write("./trees/f-snaq-genetrees.nwk")

<svg class="toyplot-canvas-Canvas" height="300.0px" id="tdf66c72be5ca46c99511997807fd3916" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 260.0 300.0" width="260.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0.477 r1 r0 r2 r3 r4 r5 r6 r7 0 1 3

### Infer ASTRAL tree from inferred gene trees

In [29]:
# infer MSC sptree with ASTRAL
ast = ipa.astral(
    data=[i.write() for i in toytree.mtree("./trees/f-genetrees.nwk")],
    name="f-astral-genetrees",
)
ast.run()

# draw sptree
atree = toytree.tree(ast.tree).root("r7")
atree.draw(**ts);

# save tree file
atree.write("./trees/f-astral-genetrees.nwk")

[astral.5.7.1.jar]
inferred tree written to (/home/deren/gentime-attraction/notebooks/analysis-astral/f-astral-genetrees.tre)


<svg class="toyplot-canvas-Canvas" height="300.0px" id="t5c24d331cef745958afc601c79d38426" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 260.0 300.0" width="260.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r1 r0 r2 r3 r4 r5 r6 r7 0 1 2